# Vehicle Choice Mixed Logit

In [ ]:
import pandas as pd

import larch as lx
from larch import PX

In this example, we will demonstrate the estimatation of parameters for
mixed logit models.  The examples are based on a problem set originally published by Ken Train.

To ensure good preformance and ease of use, estimating parameters for a mixed logit model in Larch requires the `jax` library as the compute engine.  Importing it here allows us to confirm that it is available.

In [ ]:
import jax  # noqa: F401

The data represent consumers' choices among vehicles in stated preference experiments. The data are from a study that he did for Toyota and GM to assist them in their analysis of the potential marketability of electric and hybrid vehicles, back before hybrids were introduced.   

In each choice experiment, the respondent was presented with three vehicles, with the price and other attributes of each vehicle described. The respondent was asked to state which of the three vehicles he/she would buy if the these vehicles were the only ones available in the market. There are 100 respondents in our dataset (which, to reduce estimation time, is a subset of the full dataset which contains 500 respondents.) Each respondent was presented with 15 choice experiments, and most respondents answered all 15. The attributes of the vehicles were varied over experiments, both for a given respondent and over respondents. The attributes are: price, operating cost in dollars per month, engine type (gas, electric, or hybrid), range if electric (in hundreds of miles between recharging), and the performance level of the vehicle (high, medium, or low). The performance level was described in terms of top speed and acceleration, and these descriptions did not vary for each level; for example, "High" performance was described as having a top speed of 100 mpg and 12 seconds to reach 60 mpg, and this description was the same for all "high" performance vehicles. 

In [ ]:
raw_data = pd.read_csv(
    "data.txt",
    names=[
        "person_id",
        "case_id",
        "chosen",
        "price",  # dollars
        "opcost",  # dollars per month
        "max_range",  # hundreds of miles (0 if not electric)
        "ev",  # bool
        "gas",  # bool
        "hybrid",  # bool
        "hiperf",  # High performance (bool)
        "medhiperf",  # Medium or high performance (bool)
    ],
    sep=r"\s+",
)
raw_data

Create `alt_id` values for each row within each group, and set
the index on case and alternative id's.

In [ ]:
raw_data["alt_id"] = raw_data.groupby("case_id").cumcount() + 1
raw_data = raw_data.set_index(["case_id", "alt_id"])
raw_data["price_scaled"] = raw_data["price"] / 10000
raw_data["opcost_scaled"] = raw_data["opcost"] / 10
data = lx.Dataset.construct.from_idca(raw_data)

## Simple MNL

Start with a simple MNL model, using all the variable in the dataset without transformations.

In [ ]:
simple = lx.Model(data)
simple.utility_ca = (
    PX("price_scaled")
    + PX("opcost_scaled")
    + PX("max_range")
    + PX("ev")
    + PX("hybrid")
    + PX("hiperf")
    + PX("medhiperf")
)
simple.choice_ca_var = "chosen"
simple.maximize_loglike(stderr=True, options={"ftol": 1e-9});

In [ ]:
# TEST
from pytest import approx

assert simple.most_recent_estimation_result["loglike"] == approx(-1399.1932)

expected_value = {
    "price_scaled": -0.4167,
    "opcost_scaled": -0.1288,
    "max_range": 0.4770,
    "ev": -1.3924,
    "hybrid": 0.3555,
    "hiperf": 0.1099,
    "medhiperf": 0.3841,
}
assert simple.parameters["value"].to_series().to_dict() == approx(
    expected_value, rel=5e-2
)

expected_stderr = {
    "price_scaled": 0.0332,
    "opcost_scaled": 0.0353,
    "max_range": 0.1765,
    "ev": 0.2766,
    "hybrid": 0.1218,
    "hiperf": 0.0838,
    "medhiperf": 0.0855,
}
assert simple.parameters["std_err"].to_series().to_dict() == approx(
    expected_stderr, rel=2e-3
)

## Mixture Model

To create a mixed logit model, we can start with a copy of the simple model.

In [ ]:
mixed = simple.copy()

We assign mixtures to various parameters by providing a list of `Mixture` definitions.

In [ ]:
mixed.mixtures = [
    lx.mixtures.Normal(k, f"s_{k}")
    for k in ["opcost_scaled", "max_range", "ev", "hybrid", "hiperf", "medhiperf"]
]

The estimation of the mixed logit parameters sometimes performs
poorly (i.e. may prematurely call itself converged) if you start
from the MNL solution, so it can be advantageous to start from 
the null parameters instead.

In [ ]:
mixed.pvals = "null"

There are several extra settings that can be manipulated on the
mixed logit model, including the number of draws and a random seed
for generating draws.

In [ ]:
mixed.n_draws = 200
mixed.seed = 42

In [ ]:
mixed.maximize_loglike(stderr=True, options={"ftol": 1e-9});

In [ ]:
# TEST
assert mixed.most_recent_estimation_result["loglike"] == approx(-1385.494141)

expected_value = {
    "ev": -3.032238243376885,
    "hiperf": 0.1763708168391289,
    "hybrid": 0.5461971243787561,
    "max_range": 0.8886978044065789,
    "medhiperf": 0.7325188113437849,
    "opcost_scaled": -0.2198851109606838,
    "price_scaled": -0.6478517959040359,
    "s_ev": -2.804510536354318,
    "s_hiperf": -0.5218085534690989,
    "s_hybrid": 1.1432535820795153,
    "s_max_range": -0.13064214289199153,
    "s_medhiperf": 1.5746533061378736,
    "s_opcost_scaled": 0.5204752418536094,
}

assert mixed.parameters["value"].to_series().to_dict() == approx(
    expected_value, rel=5e-2
)

expected_stderr = {
    "ev": 0.7801142930984497,
    "hiperf": 0.12483992427587509,
    "hybrid": 0.19360417127609253,
    "max_range": 0.32432201504707336,
    "medhiperf": 0.2088639885187149,
    "opcost_scaled": 0.07158942520618439,
    "price_scaled": 0.10207334160804749,
    "s_ev": 0.7572864890098572,
    "s_hiperf": 0.8733952045440674,
    "s_hybrid": 0.6600803732872009,
    "s_max_range": 0.9942808151245117,
    "s_medhiperf": 0.6126551628112793,
    "s_opcost_scaled": 0.18554426729679108,
}
assert mixed.parameters["std_err"].to_series().to_dict() == approx(
    expected_stderr, rel=5e-2
)

## Panel Data

In the mixed logit model above, we have ignored the fact that 
we have "panel data", where multiple observations are made by the
same decision maker who (presumably) has stable preferences with
respect to the choice attributes.  We can tell Larch which data
column represents the panel identifiers in the `groupid` attribute,
(which we set to `"person_id"`) and then
use that information to build a significantly better model.

In [ ]:
panel = mixed.copy()
panel.groupid = "person_id"
panel.maximize_loglike(stderr=True, options={"ftol": 1e-9});

In [ ]:
# TEST
assert panel.most_recent_estimation_result["loglike"] == approx(-1330.442871)

expected_value = {
    "ev": -1.7034994106596084,
    "hiperf": 0.09213393502033553,
    "hybrid": 0.46842352186376457,
    "max_range": 0.4627547334553541,
    "medhiperf": 0.5090292596924697,
    "opcost_scaled": -0.1388731660486894,
    "price_scaled": -0.5014897667296647,
    "s_ev": -0.921130570903971,
    "s_hiperf": -0.4193628031632754,
    "s_hybrid": 0.8446756225312494,
    "s_max_range": 0.7102007254766374,
    "s_medhiperf": 0.6029327016972847,
    "s_opcost_scaled": -0.3470455945572811,
}
assert panel.parameters["value"].to_series().to_dict() == approx(
    expected_value, rel=5e-2
)

expected_stderr = {
    "ev": 0.3431136906147003,
    "hiperf": 0.10426269471645355,
    "hybrid": 0.16409245133399963,
    "max_range": 0.22775666415691376,
    "medhiperf": 0.11354632675647736,
    "opcost_scaled": 0.052873723208904266,
    "price_scaled": 0.039680227637290955,
    "s_ev": 0.22132474184036255,
    "s_hiperf": 0.14358796179294586,
    "s_hybrid": 0.133216992020607,
    "s_max_range": 0.18694375455379486,
    "s_medhiperf": 0.14173047244548798,
    "s_opcost_scaled": 0.04932473972439766,
}
assert panel.parameters["std_err"].to_series().to_dict() == approx(
    expected_stderr, rel=5e-2
)

We can review summary statistics about the distibution of the mixed parameters:

In [ ]:
panel.mixture_summary()

In [ ]:
# TEST
assert panel.mixture_summary()["mean"].to_dict() == approx(
    {
        "ev": -1.703514575958252,
        "hiperf": 0.09218784421682358,
        "hybrid": 0.46854260563850403,
        "max_range": 0.4628429114818573,
        "medhiperf": 0.5090671181678772,
        "opcost_scaled": -0.13884519040584564,
    },
    rel=1e-2,
)
assert panel.mixture_summary()["share +"].to_dict() == approx(
    {
        "ev": 0.032249998301267624,
        "hiperf": 0.5868499875068665,
        "hybrid": 0.7102000117301941,
        "max_range": 0.7425000071525574,
        "medhiperf": 0.800599992275238,
        "opcost_scaled": 0.3444499969482422,
    },
    rel=1e-2,
)

The "share" columns show the share of random simulations with positive, negative, or 
zero values for each model parameter. This summary shows that about a third of people 
show a positive parameter on operating costs, i.e. they prefer higher costs.  That's 
probably not reasonable.

## Log-Normal Parameter

Let's change the operating cost parameter to have a negative log-normal distribution.

In [ ]:
panel2 = panel.copy()
panel2.mixtures[0] = lx.mixtures.NegLogNormal("opcost_scaled", "s_opcost_scaled")
panel2.pvals = "null"
panel2.maximize_loglike(stderr=True, options={"ftol": 1e-9});

In [ ]:
panel2.mixture_summary()

In [ ]:
panel2.mixture_density("opcost_scaled");

In [ ]:
panel2.mixture_density("hybrid");